# Spatial and spectral resolutions.

* **Special requirements:** A Google account, access to Google Earth Engine.
* **Prerequisites:** You should have completed the `2.1_ENGN3903_Satellite images and bands`, the `2.2 ENGN3903_Images, collections, and filters` notebook.


## Background

> Note: Refer to Lecture 2 of week 2 for the relevant content for this lab

Resolution is the ability to discriminate information in an image. The discrimination of information refers not only to the spatial detail (spatial resolution) but also to the number of spectral wavebands, their bandwidths, and spectral cover (spectral resolution), the temporal frequency of observations (temporal resolution) and the signal to noise ration or its ability to distinguish vitiations in the energy detected (radiomatric resolution). One may also have to consider the sensor's potential to acquire information at different viewing angles (angular resolution) or polarization channels (polimetric resolution).


## Aims of the practical session

This practical has three aims:
1. To understand the differences between the spatial and spectral resolutions of different satellite sensors.
2. To plot the spectral signature of different land cover types, and
3. Learn how to use band-combinations to highlight different landscape features 

***

## Description

In this notebook we'll load images from 3 different sensors and will use them to understand the differences between the spatial and spectral and resolution in the remote sensing context. 

First we will:
- Load Landsat, Sentinel 2, and MODIS images for the Canberra region.

Then we will:
- visualize the images of the different sensors side-by-side to understand the concept of spatial resolution;
- Compare the spectral resolution (how many 'bands') of different sensors;
- Apply various band-combinations to Landsat images to highlight different landscape features.

<div class="alert alert-block alert-warning">
<b>Assessment:</b> Once you finish the practical and the excercises, remember to submit your notebook through Wattle by Sunday.
Challenges are optional and will not be part of the assessment.
</div>

***

## Getting started


### Load packages

Import Python packages that are used for the analysis.


In [1]:
# %matplotlib inline
import geemap as gmap
import os
import ee
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Connect to Google Earth Engine (GEE)

Connect to the GEE so we can access GEE datasets and computing assets.
You may be required to input your Google account name and password. Please keep those safe and don't share them with anyone.

In [2]:
m = gmap.Map()

***

## Comparing the spatial resolution of different sensors.

Load the `MODIS` and `Sentinel-2` image collections that intersect the ACT and visualize them in a map

> Note, to 'clip' a MODIS imageCollection to a polygon, we can't rely on the syntax `.filterBounds()`, instead we need to map the `image.clip` function over the time-series. ie. `modis.map(lambda image: image.clip(polygon))`

In [3]:
# First, let's create a polygon around the ACT
act = ee.Geometry.Polygon([[148.7392751586051,-36.011462319908816],
   [149.8598806273551,-36.011462319908816],
   [149.8598806273551,-35.1087997777942],
   [148.7392751586051,-35.1087997777942],
   [148.7392751586051,-36.011462319908816]])

# Let's add the polygon to the map
Map = gmap.Map(center=[-35.2041, 149.2721], zoom=10)
Map.addLayer(act,{},'act')

### A quick note on image composites
In the code below, we filter the datasets to a time-range `.filterDate('2019-02', '2019-04')`, this will give us a two-month long time-series of satellite images over that region. We then _reduce_ that time-series (collapse the time-dimension) with the function `.median()`, which will create what Earth Observation scientists call a 'composite image', that is, an image that is representative of all images over a given time. Individual remote sensing images can be affected by noisy data, including clouds, cloud shadows, and haze. To produce cleaner images that can be compared more easily across time, we can create 'summary' images or 'composites' that combine multiple images into one. Median summaries are an extremely useful and very commonly used approach to summarising time-series of satellite images.

In [4]:
# Now let's add the image collections.
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
modis = ee.ImageCollection("MODIS/061/MYD09A1").select([
    'sur_refl_b01',
    'sur_refl_b02',
    'sur_refl_b03',
    'sur_refl_b04',
    'sur_refl_b05',
    'sur_refl_b06',
    'sur_refl_b07'])

# Filter the image collections by date, and by location, then reduce time dim using median
s2 = s2.filterDate('2019-02', '2019-04').map(lambda image: image.clip(act)).median()
modis = modis.filterDate('2019-02', '2019-04').map(lambda image: image.clip(act)).median()

# Lastly, let's give some visualization paramaters to each collection.
s2VisParam = {'bands': ["B4","B3","B2"],
              'max': 2700,
              'min': 0}

# Note that the visualization are different for each image collection.
modisVisParams = {'bands': ["sur_refl_b01","sur_refl_b04","sur_refl_b03"],
              'max': 2100,
              'min': 0}
              
Map.addLayer(s2, s2VisParam, 's2')
Map.addLayer(modis, modisVisParams, 'modis')
Map.addLayerControl()

Map

Map(center=[-35.2041, 149.2721], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

### <a name="ex1"></a> Exercise 1 - Add a `Landsat 8` image collection to the map.

You've done this before, just make sure you filter the image collection with the same dates as the ones above, and reduce the time-series using the `.median()` function


In [5]:
# Your code goes here


### <a name="ex1"></a> Exercise 2 - Understanding spatial resolution


<div class="alert alert-block alert-danger">

**Answer the following questions:**

1) What is the smallest feature you can identify using the MODIS, Landsat, and Sentinel 2 imagery?
    
2) Which satellite sensor has the best and worst spatial resolution.

</div>    

Try identifying the following features / places / landmarks:
>- Telstra Tower
>- a car
>- a football/cricket field
>- Canberra Airport
>- Lake George,
>- a crop field
>- a mountain range or a large forest.



Your answer goes here:

.

***

## Comparing the spectral resolution of different sensors

There are few hyperspectral sensors in orbit.

Fortunately, we have access to [Hyperion](https://www.usgs.gov/centers/eros/science/earth-observing-1-eo-1) data. Hyperion was a hyperspectral sensor that was Decommissioned in 2017, but gathered data in many places around the globe.

As with the other sensors, we can add these data to our map, and filter by date and location.


> **Caution** Because Hyperion only provides irradiance data, we might have to 1) either perform atmospheric correction, or 2) load TOA Landsat data and uncorrected MODIS so the plots are comparable

> Remember, whenever we load Landsat Collection2, we need to rescale the digital numbers to surface-reflectance

In [6]:
def rescale_landsatC2(image):
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    # Replace the original bands with the scaled ones
    return image.addBands(opticalBands, None, True)

In [9]:
Map2 = gmap.Map(center=[-35.2041, 149.2721], zoom=9)

# Geth the Hyperion image collection and a Landsat image to display
hyper = ee.ImageCollection("EO1/HYPERION").map(lambda image: image.clip(act)).median()

# load Landsat but only load the optical bands
clearC2 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_090085_20210118') \
        .select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7',]).clip(act)
clearC2 = rescale_landsatC2(clearC2)

# Set the visualization parameters for the images we're going to display
hyperVisParams = {'bands': ["B035","B023","B015"],
                 'min':500,
                 'max':6000}

landsatC2_vis = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
              'min': 0,
              'max': 0.4}

# Add the layers to the map
Map2.addLayer(modis, modisVisParams, 'MODIS image collection')
Map2.addLayer(clearC2, landsatC2_vis,' Landsat image')
Map2.addLayer(hyper, hyperVisParams, 'Hyperion hyperspectral image collection')
Map2.addLayerControl()
Map2.set_plot_options(add_marker_cluster=True)
Map2

Map(center=[-35.2041, 149.2721], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

### <a name="ex3"></a> Exercise 3 - Understanding the spectral resolution.


<div class="alert alert-block alert-danger">

**Compare and contrast:**

1) The spatial *coverage* of the MODIS data, versus the Hyperion data
    
2) The spectral resolution of the MODIS, Landsat, and Hyperion images.

</div>    


### Displaying the spectral signatures of different land covers

Now click on the map toolbar and select the 'Plotting' icon as shown below. Pay attention to the image selected for this excercise. If you add more data to the map, you'll need to select the image you want to explore.

![3.1_fig3.PNG](../figures/3.1_fig3.PNG)

Now click anywhere on the image and look at the spectral signature of the selected feature. Is it what you would expect?

In [8]:
Map3 = gmap.Map(center=[-35.9659, 149.4965], zoom=8)

# Geth  a Landsat image to display
clearC2 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_090085_20210118') \
        .select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7',])
clearC2 = rescale_landsatC2(clearC2)

# Add the layers to the map
Map3.addLayer(clearC2, landsatC2_vis,' Landsat image')
Map3.addLayerControl()
Map3.set_plot_options(add_marker_cluster=True)
Map3

Map(center=[-35.9659, 149.4965], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

Now we can export these points and the corresponding band values to a CSV and Shapefile to use them in another software (e.g. QGIS) if we wanted to.

In [ ]:
# Change the path to the 'Downloads' folder in your computer
folder = 'C:/User/Downloads/'
points_shp = os.path.join(folder, 'points.shp')
Map3.extract_values_to_points(os.path.join(folder, 'points4.shp'))

For us it is important to know the values of each band for each of the points/features we just created and we want to compare them.

   
To do his, we need to:
1. Click on the map tools icon (the little wrench), and select 'Collect Training samples'.
2. A table will appear. Fill the 'Required Property' with `landCover`, and the 'Integer Value' with `1`.
3. Then fill the 'Optional Property' with `label`, and the 'String Value' with `forest`. Click `Apply` and then collect ~5 points over forested areas in the image using the 'Draw a Marker' tool on the left hand side of the map.
4. Having done this, change the 'Integer Value' to `2`, and the 'String Value with `crop`. Click `Apply` and collect ~5 points over forests.
5. continue until you have collected data for at least three different land covers (e.g. forest, clouds, city)

> Note: Make sure you **add points where all three datasets coincide**

![collectPts](../figures/3.1_fig4_collectPts2.JPG)

In [ ]:
Map4 = gmap.Map(center=[-35.9659, 149.4965], zoom=8)

# Get  a Landsat image to display
clearC2 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_090085_20210118') \
        .select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7',])
clearC2 = rescale_landsatC2(clearC2)

# Add the layers to the map
Map4.addLayer(modis, modisVisParams, 'MODIS image collection')
Map4.addLayer(clearC2, landsatC2_vis,' Landsat image')
Map4.addLayer(hyper, hyperVisParams, 'Hyperion hyperspectral image collection')
Map4.addLayerControl()
Map4.set_plot_options(add_marker_cluster=True)
Map4

### Extract band values for each pixel in the samples collected

In [ ]:
pointsLandsat = clearC2.sampleRegions(**
                               {'collection': Map4.user_rois,
                                'scale': 30,
                                'geometries': True,
                               'tileScale': 8}
                               )

# Now let's do the same with the MODIS images
pointsModis = modis.sampleRegions(**
                               {'collection': Map4.user_rois,
                                'scale': 500,
                                'geometries': True,
                               'tileScale': 8}
                               )
# Now let's do the same with the Hyperion images
pointsHyperion = hyper.sampleRegions(**
                               {'collection': Map4.user_rois,
                                'scale': 30,
                                'geometries': True,
                               'tileScale': 8}
                               )


<div class="alert alert-block alert-warning">
If you get an error like this one:
    `EEException: Parameter 'collection' is required.`
it means that you have not gathered the points. Go back to Map4, collect the training points and try again.
</div>

Now each point has the values for each Landsat band, MODIS and Hyperion band, we can export the points with the band values to a 'dataframe' (a table)

In [ ]:
df_landsat = gmap.ee_to_pandas(pointsLandsat, verbose=True)
df_modis = gmap.ee_to_pandas(pointsModis, verbose=True)
df_hyperion = gmap.ee_to_pandas(pointsHyperion, verbose=True)

# Now we'll sort the 'label' column to make the figure below look a bit better.
# Here we're assuming that you named the 'Optional Property' as 'label'
df_landsat = df_landsat.sort_values(by = ['label'])
df_modis = df_modis.sort_values(by = ['label'])
df_hyperion = df_hyperion.sort_values(by = ['label'])

# and we re-oder the columns. See https://lpdaac.usgs.gov/products/myd09a1v061/ to understand the order.
modisColumnOrder = ['sur_refl_b03','sur_refl_b04','sur_refl_b01','sur_refl_b02',
 'sur_refl_b05','sur_refl_b06','sur_refl_b07','color', 'landCover', 'label',]
df_modis = df_modis[modisColumnOrder]
df_modis.head(5)

<div class="alert alert-block alert-warning">
Did you get a `KeyError: 'label'`?
    
If so, double check if you named the 'Optional Property' as 'label', of if you used a different name. Change the code in the cell above to reflect the optional property you chose.
If you didn't get the error, you can ignore this message.
</div>

Plotting the ~190 bands of Hyperion is a challenge. So we need to make some adjustments to the dataset so it plots neatly

In [11]:
# First, we create a list of the column names. In this case, each column name is the name of each spectral band.
hyperColNames = df_hyperion.drop(['color','landCover','label'], axis=1).columns.to_list()

# and sort the band names from 
hyperColNames.sort()

# and select every 3rd band for the plot
bandLabels = hyperColNames[::3]

### Plot the spectral signatures of each sensor

In [ ]:
fig, axes = plt.subplot_mosaic(
    [['a)', 'b)'], 
     ['c)', 'c)'],
], constrained_layout=True, figsize=(15,7))

# We plot the data on each row independently
for index, row in df_modis.drop(['color','landCover','label'], axis=1).iterrows():
    axes['a)'].plot(row, color='grey' )
    n +=1

n=0
for index, row in df_landsat.drop(['color','landCover','label'], axis=1).iterrows():
    axes['b)'].plot(row, color='grey'  )
    n +=1    

n=0
for index, row in df_hyperion.drop(['color','landCover','label'], axis=1).sort_index(axis=1).iterrows():
    axes['c)'].plot(row, color='grey' , alpha=0.5 )
    n +=1 

    # Set the lables for the axes    
axes['a)'].set_ylabel('Landsat reflectance')
axes['b)'].set_ylabel('MODIS reflectance')
axes['c)'].set_ylabel('Hyperion radiance')
axes['a)'].set_xlabel('Landsat bands')
axes['b)'].set_xlabel('MODIS bands')
axes['c)'].set_xlabel('Hyperion bands')
axes['a)'].set_title('Landsat data')
axes['b)'].set_title('Modis data')
axes['c)'].set_title('Hyperion data')
plt.xticks(bandLabels, labels = bandLabels, rotation=45, fontsize=10);


***

## Band Combinations

The previous exercise showed you that features in an image look different in different bands. Some bands are better to detect the presence (or absence) of a feature.
For example, the Near Infrared (NIR) and Short-Wave Infrared (SWIR) bands are very useful to detect vegetation, and water. We can combine these bands to create 'color composites' (a.k.a false color images). A false color image is used to reveal or enhance features otherwise invisible or poorly visible to a human eye.

![false%20color.png](../figures/false%20color.png)

**What do the various landscape features (cities, lakes, forests) look like using this band combination? Why do you think some features are brighter than others? Which features are darkest?**

In [ ]:
Map6 = gmap.Map(center=[-35.9659, 149.4965], zoom=8)

# Geth  a Landsat image to display
clearC2 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_090085_20210118') \
        .select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7',])

clearC2 = rescale_landsatC2(clearC2)

# Select the bands we want to display for the false color composite
falseColor = {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'min':0, 'max':0.4}
trueColor_vis = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min': 0,'max': 0.4}

# Add the layers to the map
Map6.addLayer(clearC2,  falseColor, "False Color NIR,R,G")
Map6.addLayer(clearC2, trueColor_vis,'True Colour R,G,B')
Map6.addLayerControl()
Map6

### <a name="ex2"></a> Exercise 3 -  False color images

It is your turn to create some false color images.

Create false color images using the following bands, and answer the questions below:

- [SWIR1, NIR, Green]
- [SWIR2, SWIR2, Red]
- [SWIR1, NIR, Blue]
- [NIR, SWIR1, Red]

<div class="alert alert-block alert-danger">
NEED TO REVIEW THIS QUESTIONS

**Answer the following questions:**
>- Which band detect the presence of vegetation? why?
>- Which bands would you use to detect bare soil? why?
>- Does water look the same in all spectral bands?
>- Which bands would you use to detect water?
>- Which bands or band combination is best for detecting clouds and cloud shadows?
>- There are many bodies of water in this image (Lakes, rivers, the sea), and they all look slightly diffetent to one another. Can you explain why they look so different?

</div>

In [ ]:
# Your code goes here - [SWIR1, NIR, Green]


In [ ]:
# Your code goes here - [SWIR2, SWIR2, Red]


In [ ]:
# Your code goes here - [SWIR1, NIR, Blue]


In [ ]:
# Your code goes here - [NIR, SWIR1, Red]


**The answers to the questions go here.**


.

***

## Summary

In this notebook you have learned about how different sensors have different spatial and spectral resolutions.  The attributes of specific satellite sensors **will** affect your analysis, so you'll have to understand which sensor is better for which analysis.  Lastly, we also considered how different band combinations can be used to visualise features in the landscape. 

***

## References and useful readings
<div class="alert alert-block alert-danger">
    
**update**
    
- See recomended readings for week 2 in wattle
- https://geemap.org/
- http://dx.doi.org/10.1016/j.rse.2015.11.032
- https://doi.org/10.3390/rs1030184
- https://doi.org/10.1016/j.rse.2014.02.001
    </div>

***

## Additional information

**Sources:** 

**License:** The code in this notebook is licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/). 

**Contact:** If you need assistance, please post a question on the ENGN3903 Wattle (**check**) site 

**Last modified:** August 2022

***